In [29]:
import networkx as nx
import torch
import torch_geometric
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [162]:
cft_truth_graph = torch.load("events/event000047441-graph.pyg")

In [174]:
print("Number of CFT truth nodes:", len(cft_truth_graph.x))

cft_truth_edges = pd.DataFrame(cft_truth_graph.track_edges.T.numpy(), columns=['source', 'target'])#.reset_index()
print("Number of CFT truth edges:", len(cft_truth_edges))

particle_selections = (
    (cft_truth_graph.pt >= 1000.) 
    & (cft_truth_graph.eta > -4) 
    & (cft_truth_graph.eta < 4) 
    & (cft_truth_graph.nhits >= 3) 
    & (cft_truth_graph.primary == True) 
    & (abs(cft_truth_graph.pdgId) != 11) 
    & (cft_truth_graph.radius < 260.) 
    # & (cft_truth_graph.redundant_split_edges == False)
    # & (cft_truth_graph.barcode < 200000)
    # & (cft_truth_graph.subevent == 0)
).numpy()

selected_cft_truth_edges = cft_truth_edges[particle_selections]
selected_cft_truth_unsplit_edges = cft_truth_edges[
    particle_selections & (cft_truth_graph.redundant_split_edges == False).numpy()]

print("Number of selected CFT truth nodes:", len(np.unique(np.concatenate([selected_cft_truth_edges.source.to_numpy(), selected_cft_truth_edges.target.to_numpy()]))))
print("Number of selected CFT truth edges:", len(selected_cft_truth_edges))
print("Number of selected CFT unsplit truth nodes:", len(np.unique(np.concatenate([selected_cft_truth_unsplit_edges.source.to_numpy(), selected_cft_truth_unsplit_edges.target.to_numpy()]))))
print("Number of selected CFT unsplit truth edges:", len(selected_cft_truth_unsplit_edges))

Number of CFT truth nodes: 307435
Number of CFT truth edges: 126866
Number of selected CFT truth nodes: 15286
Number of selected CFT truth edges: 14137
Number of selected CFT unsplit truth nodes: 15186
Number of selected CFT unsplit truth edges: 13960


In [4]:
module_map_graph = nx.read_graphml("OUTPUT_DIR_GEN/event000047441_TARGET.txt")

In [179]:
module_map_nodes = pd.DataFrame(np.array(module_map_graph.nodes.data('hit_id')), columns=['node', 'hit_id']).astype({'hit_id': int})
# print("Number of nodes in input file:", len(module_map_nodes))

module_map_edges = nx.to_pandas_edgelist(module_map_graph)
module_map_edges = module_map_edges.merge(
    module_map_nodes, left_on='source', right_on='node', how='left').drop(
        ['source', 'node'], axis=1).rename(columns={'hit_id': 'source'})
module_map_edges = module_map_edges.merge(
    module_map_nodes, left_on='target', right_on='node', how='left').drop(
        ['target', 'node'], axis=1).rename(columns={'hit_id': 'target'})
# print(module_map_edges)
print("Number of module map nodes:", len(np.unique(np.concatenate([module_map_edges.source.to_numpy(), module_map_edges.target.to_numpy()]))))
print("Number of module map edges:", len(module_map_edges))

Number of module map nodes: 14924
Number of module map edges: 17211


In [178]:
module_map_truth_edges = module_map_edges[module_map_edges.is_segment_true==True]
print("Number of truth module map nodes:", len(np.unique(np.concatenate([module_map_truth_edges.source.to_numpy(), module_map_truth_edges.target.to_numpy()]))))
print("Number of truth module map edges:", len(module_map_truth_edges))

Number of truth module map nodes: 12442
Number of truth module map edges: 10131


In [180]:
# selected_cft_truth_edges
merged_truth_edges = selected_cft_truth_unsplit_edges.merge(
    module_map_truth_edges,
    on=["source", "target"],
    how='inner'
    )

print("Number of matched non-split edges:", len(merged_truth_edges))

reversed_merged_truth_edges = selected_cft_truth_unsplit_edges.merge(
    module_map_truth_edges,
    left_on=["source", "target"],
    right_on=["target", "source"],
    how='inner'
    ).rename(columns={"source_x": "source", "target_x": "target"}).drop(["source_y", "target_y"], axis=1)

print("Number of reversed non-splitmatched edges:", len(reversed_merged_truth_edges))

merged_truth_edges = pd.concat([merged_truth_edges, reversed_merged_truth_edges])

unmatched_module_map_truth_edges = module_map_truth_edges.merge(
    merged_truth_edges[["id", "source"]], on="id", how='left').rename(columns={"source_x": "source"})
unmatched_module_map_truth_edges = unmatched_module_map_truth_edges[
    unmatched_module_map_truth_edges["source_y"].isna()].drop(["source_y"], axis=1)
print("Number of unmatched module map true non-split edges:", len(unmatched_module_map_truth_edges))
unmatched_module_map_truth_edges

Number of matched non-split edges: 10026
Number of reversed non-splitmatched edges: 0
Number of unmatched module map true non-split edges: 105


,mask_edge,is_segment_true,region,id,pt_particle,source,target
310,1,1,1,e532,1.04136,22015,23719
416,1,1,1,e704,1.04136,23660,22015
1269,1,1,1,e2321,1.76169,9045,10074
1551,1,1,1,e2825,1.28945,51377,49536
1849,1,1,1,e3408,1.06264,47129,48901
...,...,...,...,...,...,...,...
9156,1,1,3,e15543,1.84472,204951,205006
9164,1,1,3,e15556,1.84472,205006,205050
9234,1,1,3,e15683,1.47904,198403,200218
9842,1,1,3,e16793,1.44852,222794,231168


In [181]:
# selected_cft_truth_edges
merged_truth_edges = selected_cft_truth_edges.merge(
    module_map_truth_edges,
    on=["source", "target"],
    how='inner'
    )

print("Number of matched edges:", len(merged_truth_edges))

reversed_merged_truth_edges = selected_cft_truth_edges.merge(
    module_map_truth_edges,
    left_on=["source", "target"],
    right_on=["target", "source"],
    how='inner'
    ).rename(columns={"source_x": "source", "target_x": "target"}).drop(["source_y", "target_y"], axis=1)

print("Number of reversed matched edges:", len(reversed_merged_truth_edges))

merged_truth_edges = pd.concat([merged_truth_edges, reversed_merged_truth_edges])

unmatched_module_map_truth_edges = module_map_truth_edges.merge(
    merged_truth_edges[["id", "source"]], on="id", how='left').rename(columns={"source_x": "source"})
unmatched_module_map_truth_edges = unmatched_module_map_truth_edges[
    unmatched_module_map_truth_edges["source_y"].isna()].drop(["source_y"], axis=1)
print("Number of unmatched module map true edges:", len(unmatched_module_map_truth_edges))
unmatched_module_map_truth_edges

Number of matched edges: 10131
Number of reversed matched edges: 0
Number of unmatched module map true edges: 0


,mask_edge,is_segment_true,region,id,pt_particle,source,target
